# AutoGluon: California House Prices

This Colab trains an AutoGluon **regression** model for the **California House Prices** competition and creates a submission CSV.

Assignment Done By :-  **Dev** **Mulchandani**

In [1]:
%%capture
!pip -q install -U pip
!pip -q install -U autogluon kaggle
print('Installed packages')

## 🔑 Kaggle API setup
1. Go to https://www.kaggle.com/account → **Create New API Token** (downloads `kaggle.json`).
2. Run the cell below and upload the file when asked.

In [2]:
from google.colab import files
import os

print('Please upload your kaggle.json (filename can be anything) ...')
uploaded = files.upload()
assert len(uploaded) == 1, 'Please upload exactly one file (kaggle.json).'
src_name = next(iter(uploaded.keys()))
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[src_name])
os.chmod('/root/.kaggle/kaggle.json', 0o600)
print('Kaggle API token installed at /root/.kaggle/kaggle.json')

Please upload your kaggle.json (filename can be anything) ...


Saving kaggle.json to kaggle.json
Kaggle API token installed at /root/.kaggle/kaggle.json


## ⚙️ Configuration
- `COMPETITION`: Kaggle slug
- `TIME_LIMIT`: Seconds to train (increase for better scores)
- `PRESETS`: AutoGluon preset (`medium_quality` is fast; `best_quality` is stronger but slower)

In [3]:
COMPETITION = 'california-house-prices'
TIME_LIMIT = 900          # 15 minutes; increase for stronger models
PRESETS = 'medium_quality'
SUBMIT = False            # set True to auto-submit to Kaggle at the end

DATA_DIR = '/content/data'
MODEL_DIR = '/content/AutoGluonModels'
import os
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
print('CONFIG OK')

CONFIG OK


## ⬇️ Download & extract data

In [4]:
import subprocess, zipfile, glob, os
print('Downloading data from Kaggle...')
subprocess.run(['kaggle', 'competitions', 'download', '-c', COMPETITION, '-p', DATA_DIR], check=True)

print('Extracting ZIP files...')
for z in glob.glob(os.path.join(DATA_DIR, '*.zip')):
    with zipfile.ZipFile(z, 'r') as zf:
        zf.extractall(DATA_DIR)
print('Data ready in', DATA_DIR)
!ls -lh /content/data

Extracting ZIP files...
Data ready in /content/data
total 116M
-rw-r--r-- 1 root root  30M Mar 19  2021 california-house-prices.zip
-rw-r--r-- 1 root root 248K Nov  2 00:02 sample_submission.csv
-rw-r--r-- 1 root root  35M Nov  2 00:02 test.csv
-rw-r--r-- 1 root root  51M Nov  2 00:02 train.csv


## 🔎 Inspect files & auto-detect columns
We read `sample_submission.csv` to determine the **ID** and **target** columns automatically.

In [5]:
from pathlib import Path
import pandas as pd

base = Path(DATA_DIR)
sample_path = next(iter(base.rglob('sample_submission*.csv')), None)
assert sample_path is not None, 'sample_submission.csv not found. Check download/extract steps.'

sample = pd.read_csv(sample_path)
print('Sample columns:', list(sample.columns))

# Infer id and target from the sample file (2 columns expected)
assert len(sample.columns) >= 2, 'Unexpected sample_submission format.'
id_col = sample.columns[0]
target_col = sample.columns[1]

train_path = next(iter(base.rglob('train.csv')))
test_path  = next(iter(base.rglob('test.csv')))
train = pd.read_csv(train_path)
test  = pd.read_csv(test_path)
print('Train shape:', train.shape)
print('Test shape:', test.shape)
print('Inferred id_col =', id_col, '| target_col =', target_col)
train.head()

Sample columns: ['Id', 'Sold Price']
Train shape: (47439, 41)
Test shape: (31626, 40)
Inferred id_col = Id | target_col = Sold Price


,Id,Address,Sold Price,Summary,Type,Year built,Heating,Cooling,Parking,Lot,...,Parking features,Tax assessed value,Annual tax amount,Listed On,Listed Price,Last Sold On,Last Sold Price,City,Zip,State
0,0,540 Pine Ln,3825000.0,"540 Pine Ln, Los Altos, CA 94022 is a single f...",SingleFamily,1969.0,"Heating - 2+ Zones, Central Forced Air - Gas","Multi-Zone, Central AC, Whole House / Attic Fan","Garage, Garage - Attached, Covered",1.0,...,"Garage, Garage - Attached, Covered",886486.0,12580.0,2019-10-24,4198000.0,NaN,NaN,Los Altos,94022,CA
1,1,1727 W 67th St,505000.0,"HURRY, HURRY.......Great house 3 bed and 2 bat...",SingleFamily,1926.0,Combination,"Wall/Window Unit(s), Evaporative Cooling, See ...","Detached Carport, Garage",4047.0,...,"Detached Carport, Garage",505000.0,6253.0,2019-10-16,525000.0,2019-08-30,328000.0,Los Angeles,90047,CA
2,2,28093 Pine Ave,140000.0,'THE PERFECT CABIN TO FLIP! Strawberry deligh...,SingleFamily,1958.0,Forced air,NaN,0 spaces,9147.0,...,NaN,49627.0,468.0,2019-08-25,180000.0,NaN,NaN,Strawberry,95375,CA
3,3,10750 Braddock Dr,1775000.0,Rare 2-story Gated 5 bedroom Modern Mediterran...,SingleFamily,1947.0,Central,Central Air,"Detached Carport, Driveway, Garage - Two Door",NaN,...,"Detached Carport, Driveway, Garage - Two Door",1775000.0,20787.0,2019-10-24,1895000.0,2016-08-30,1500000.0,Culver City,90230,CA
4,4,7415 O Donovan Rd,1175000.0,Beautiful 200 acre ranch land with several pas...,VacantLand,NaN,NaN,NaN,0 spaces,NaN,...,NaN,NaN,NaN,2019-06-07,1595000.0,2016-06-27,900000.0,Creston,93432,CA


## 🧠 Train AutoGluon (regression)

In [6]:
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

# Optional: drop heavy/free-text columns if present (speeds things up)
maybe_text_cols = [c for c in train.columns if train[c].dtype == 'object' and train[c].str.len().mean() > 60]
print("Dropping (possible long-text) cols:", maybe_text_cols)

# Drop ID and long-text columns
X_train_full = train.drop(columns=[id_col] + maybe_text_cols)
X_test = test.drop(columns=[id_col] + maybe_text_cols)

# Sample only 20% of training data
X_train_sampled, _ = train_test_split(X_train_full, train_size=0.2, random_state=42)
print(f"Using {len(X_train_sampled)} rows out of {len(X_train_full)} for training (20%)")

# Define fast hyperparameters (tree models only)
fast_hps = {
    'GBM': [{}],  # LightGBM
    'RF':  [{}],  # RandomForest
    'XT':  [{}],  # ExtraTrees
}

# Train predictor
predictor = TabularPredictor(
    label=target_col,
    path=MODEL_DIR,
    eval_metric='root_mean_squared_error'
).fit(
    X_train_sampled,
    hyperparameters=fast_hps,
    presets='medium_quality_faster_train',  # faster than medium_quality
    time_limit=600,                         # 10 minutes hard cap
    num_bag_folds=0,                        # no bagging
    num_stack_levels=0,                     # no stacking
    verbosity=2
)


Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       11.16 GB / 12.67 GB (88.1%)
Disk Space Avail:   66.96 GB / 112.64 GB (59.4%)
Presets specified: ['medium_quality_faster_train']


Dropping (possible long-text) cols: ['Summary']
Using 9487 rows out of 47439 for training (20%)


Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "/content/AutoGluonModels"
Train Data Rows:    9487
Train Data Columns: 38
Label Column:       Sold Price
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (43300000.0, 102000.0, 1280941.47497, 1625698.86487)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11433.70 MB
	Train Data (Original)  Memory Usage: 12.37 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually

## 🔮 Predict on test & build submission

In [7]:
import pandas as pd

preds = predictor.predict(X_test)

# Build submission by aligning predictions to sample ids
sub = pd.DataFrame({id_col: test[id_col].values, target_col: preds.values})

SUBMISSION_PATH = f'{DATA_DIR}/my_submission.csv'
sub.to_csv(SUBMISSION_PATH, index=False)
print('Saved submission to', SUBMISSION_PATH)
sub.head()

Saved submission to /content/data/my_submission.csv


,Id,Sold Price
0,47439,8.443386e+05
1,47440,5.039987e+05
2,47441,8.755401e+05
3,47442,8.749112e+05
4,47443,1.133098e+06


In [8]:
import pandas as pd
import numpy as np

# 1) Pick the id column robustly (case-insensitive). If none, fall back to the index.
id_col = next((c for c in test.columns if c.lower() == "id"), None)
if id_col is not None:
    ids = test[id_col].copy()
else:
    # No id column? use the row index as an id
    ids = test.index.to_series().rename("id")

# 2) Build the feature-only test frame (don’t lose the ids!)
X_test = test.drop(columns=[id_col] if id_col else [], errors="ignore")
X_test = X_test.drop(columns=maybe_text_cols, errors="ignore")

# 3) Get predictions
# For binary classification where Kaggle expects probability of the positive class:
try:
    preds = predictor.predict_proba(X_test)  # DataFrame/ndarray
    # If it's a DataFrame with class columns, take the positive class (1 or "stroke")
    if isinstance(preds, pd.DataFrame):
        pos_col = 1 if 1 in preds.columns else "stroke" if "stroke" in preds.columns else preds.columns[-1]
        preds = preds[pos_col].values
    else:
        preds = np.asarray(preds)[:, 1]
except Exception:
    # For regression or label predictions:
    preds = predictor.predict(X_test)
    preds = np.asarray(preds)

# 4) Create submission (ensure alignment with ids)
submission = pd.DataFrame({
    "id": ids.values,
    "stroke": preds
})

out_path = "my_submission_ca.csv"
submission.to_csv(out_path, index=False)
print("✓ Submission saved to:", out_path)
display(submission.head())



✓ Submission saved to: my_submission_ca.csv


,id,stroke
0,47439,8.443386e+05
1,47440,5.039987e+05
2,47441,8.755401e+05
3,47442,8.749112e+05
4,47443,1.133098e+06


## 🚀 (Optional) Submit to Kaggle
Set `SUBMIT=True` in the config cell first.

In [11]:
import subprocess, os
if SUBMIT:
    assert os.path.exists(SUBMISSION_PATH), 'Submission CSV not found.'
    print('Submitting to Kaggle leaderboard...')
    subprocess.run(['kaggle', 'competitions', 'submit', '-c', COMPETITION, '-f', SUBMISSION_PATH, '-m', 'AutoGluon regression'], check=True)
    print('Submission sent! Check Kaggle for your score.')
else:
    print('SUBMIT is False — skipping submission. Set SUBMIT=True to submit.')

SUBMIT is False — skipping submission. Set SUBMIT=True to submit.
